In [11]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV


In [12]:
sheet_name='Sheet3'
data= pd.read_excel("F:/Thesis/Nazmul sir/DATA/bangladesh-inflation-rate-cpi.xlsx", sheet_name=sheet_name)
print(data.head())
""" plt.plot(data['Year'],data['Exchange rate'],'o-')
plt.show() """

        Date  Exchange rate
0 1996-10-09          42.45
1 1996-10-10          42.45
2 1996-10-11          42.45
3 1996-10-14          42.45
4 1996-10-15          42.45


" plt.plot(data['Year'],data['Exchange rate'],'o-')\nplt.show() "

In [13]:
for i in range(1,8):
    data[f'Lag{i}']=data['Exchange rate'].shift(i)
data.dropna(inplace=True)
data['Day_of_Week'] = data['Date'].dt.day
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year
print(data.head())

         Date  Exchange rate   Lag1   Lag2   Lag3   Lag4   Lag5   Lag6   Lag7  \
7  1996-10-18          42.45  42.45  42.45  42.45  42.45  42.45  42.45  42.45   
8  1996-10-21          42.45  42.45  42.45  42.45  42.45  42.45  42.45  42.45   
9  1996-10-22          42.45  42.45  42.45  42.45  42.45  42.45  42.45  42.45   
10 1996-10-23          42.45  42.45  42.45  42.45  42.45  42.45  42.45  42.45   
11 1996-10-24          42.45  42.45  42.45  42.45  42.45  42.45  42.45  42.45   

    Day_of_Week  Month  Year  
7            18     10  1996  
8            21     10  1996  
9            22     10  1996  
10           23     10  1996  
11           24     10  1996  


In [14]:
x=data.drop(columns=['Date','Exchange rate','Year'])
y=data['Exchange rate']
print(x.head())
print(y.head())
print(y.dtype)



     Lag1   Lag2   Lag3   Lag4   Lag5   Lag6   Lag7  Day_of_Week  Month
7   42.45  42.45  42.45  42.45  42.45  42.45  42.45           18     10
8   42.45  42.45  42.45  42.45  42.45  42.45  42.45           21     10
9   42.45  42.45  42.45  42.45  42.45  42.45  42.45           22     10
10  42.45  42.45  42.45  42.45  42.45  42.45  42.45           23     10
11  42.45  42.45  42.45  42.45  42.45  42.45  42.45           24     10
7     42.45
8     42.45
9     42.45
10    42.45
11    42.45
Name: Exchange rate, dtype: float64
float64


In [15]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=GradientBoostingRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)
grid_search.fit(x, y)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Initialize list to store validation RMSE
val_rmse_list = []
val_r2=[]

# Initialize KFold
kf = KFold(n_splits=100, shuffle=True, random_state=42)
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=.8,random_state=42)

# Perform k-fold cross-validation
for train_index, val_index in kf.split(x_train):
    x_train_fold, x_val_fold = x_train.iloc[train_index], x_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Create and train RandomForestRegressor with best hyperparameters
    gb_model = GradientBoostingRegressor(**best_params, random_state=42)
    gb_model.fit(x_train_fold, y_train_fold)
    
    # Make predictions on validation set
    y_pred_val = gb_model.predict(x_val_fold)

    # Calculate validation RMSE
    val_rmse = mean_squared_error(y_val_fold, y_pred_val, squared=False)
    val_rmse_list.append(val_rmse)
    r2=r2_score(y_val_fold,y_pred_val)
    val_r2.append(r2)

# Calculate average model RMSE
mean_rmse = np.mean(val_rmse_list)

# Print results
print("Average model RMSE:", mean_rmse)
print (best_params)
print(f"Average r2 score is {np.mean(val_r2)}")


Average model RMSE: 0.29796591488771546
{'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 150}
Average r2 score is 0.9995130401133255


In [16]:
Final_model= GradientBoostingRegressor(**best_params,random_state=42)
Final_model.fit(x_train,y_train)
Prediction=Final_model.predict(x_test)
RMSE=mean_squared_error(Prediction,y_test,squared=False)
print(f'Final model RMSE: {RMSE}')

Final model RMSE: 0.3196246868647156
